In [1]:
import numpy as np

In [2]:
class Bee:
    def __init__(self, posicion, fitness):
        self.posicion = posicion
        self.fitness = fitness


class ABC:
    def __init__(
        self,
        funcion_objetivo: list,
        colony_size: int = 50,
        max_envios: int = 100,
        limit: int = 50,
        nv: int = 2,
    ):
        """Algoritmo de minimizacion ABC

        Args:
            funcion_objetivo (list): funcion objetivo
            colony_size (int, optional): tamano de la colonia. Defaults to 30.
            max_envios (int, optional): numero de iteraciones. Defaults to 100.
            nv (int, optional): numero de variables de la funcion objetivo. Defaults to 2.
        """
        self.funcion_objetivo = funcion_objetivo
        self.colony_size = colony_size
        self.max_envios = max_envios
        self.nv = nv

    def enviar_abejas_obreras(self):
        for i in range(self.colony_size):
            nueva_posicion = self.poblacion[i].posicion + np.random.uniform(
                -1, 1, size=self.nv
            )
            nuevo_fitness = self.funcion_objetivo(*nueva_posicion)
            if (
                self.poblacion[i].fitness is None
                or nuevo_fitness < self.poblacion[i].fitness
            ):
                self.poblacion[i].posicion = nueva_posicion
                self.poblacion[i].fitness = nuevo_fitness

    def enviar_abejas_observadoras(self):
        total_fitness = sum(bee.fitness for bee in self.poblacion)
        probabilidades = [bee.fitness / total_fitness for bee in self.poblacion]

        for i in range(self.colony_size):
            elegir_index = np.random.choice(range(self.colony_size), p=probabilidades)
            nueva_posicion = self.poblacion[elegir_index].posicion + np.random.uniform(
                -1, 1, size=self.nv
            )
            nuevo_fitness = self.funcion_objetivo(*nueva_posicion)
            if nuevo_fitness < self.poblacion[elegir_index].fitness:
                self.poblacion[elegir_index].posicion = nueva_posicion
                self.poblacion[elegir_index].fitness = nuevo_fitness

    def enviar_abejas_exploradoras(self):
        for i in range(self.colony_size):
            if self.poblacion[i].fitness is None:
                self.poblacion[i].posicion = np.random.uniform(-10, 10, size=self.nv)
                self.poblacion[i].fitness = self.funcion_objetivo(
                    *self.poblacion[i].posicion
                )

    def ejecutarABC(self):
        self.poblacion = [
            Bee(np.random.uniform(-10, 10, size=self.nv), None)
            for _ in range(self.colony_size)
        ]

        for _ in range(self.max_envios):
            self.enviar_abejas_obreras()
            self.enviar_abejas_observadoras()
            self.enviar_abejas_exploradoras()

        return min(self.poblacion, key=lambda bee: bee.fitness).posicion

In [3]:
# Ejecucion
def objetive_function(x, y):
    return (y - x**2) ** 2 + (1 - x) ** 2


abc = ABC(objetive_function)
mejor_solucion = abc.ejecutarABC()
print("Mejor solución encontrada:", mejor_solucion)
print(
    "Valor de la función objetivo en la mejor solución:",
    objetive_function(*mejor_solucion),
)

Mejor solución encontrada: [1.00674229 1.01586843]
Valor de la función objetivo en la mejor solución: 5.0926513476442606e-05
